In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn import metrics

In [1]:
import seaborn as sns
df = sns.load_dataset('diamonds')

In [2]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [5]:
df = pd.get_dummies(df, drop_first=True)

In [8]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [9]:
X = df.drop(columns="price")
y = df["price"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = SGDRegressor()

In [20]:
# зададим пространство поиска гиперпараметров
param_grid = {
                "loss": ["squared_error", "epsilon_insensitive"],
                "penalty": ["elasticnet"],
                "alpha": np.logspace(-3, 3, 10),
                "l1_ratio": np.linspace(0, 1, 10),
                "learning_rate": ["constant"],
                "eta0": np.logspace(-4, -1, 4)
}

grid_search = GridSearchCV(
    estimator=SGDRegressor(
        random_state=42, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
) 

#Обучаем модель
%time grid_search.fit(X_train, y_train)

#Выводим значения метрики 
y_test_pred = grid_search.predict(X_test)
print('MSE на тестовом наборе: {:.2f}'.format(metrics.mean_squared_error(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 50.8 s
Wall time: 2min 6s
MSE на тестовом наборе: 0.04
Наилучшие значения гиперпараметров: {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


In [21]:
print('MSE на тестовом наборе: {:.3}'.format(metrics.mean_squared_error(y_test, y_test_pred)))

MSE на тестовом наборе: 0.044


## Метод Ньютона

In [24]:
from sympy import symbols, diff

In [50]:
#считаем уравнение
a = 0.7
def abc(a):
    x0, x1 = symbols('x0, x1')
    f = 6*x0**5 - 5*x0**4 - 4*x0**3 + 3*x0**2
    f1 = diff(f, x0)
    x1 = x0 - f/f1
    return round(x1.subs(x0, a).evalf(), 4)
while a != abc(a):
    abc(a)
    a = abc(a)
    print(a.round(3))   


0.630
0.629


In [51]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

In [52]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)


21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [53]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [56]:
#оптимизируем с помощью метода Ньютона
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

In [58]:
#считаем уравнение
a = 12
def abc(a):
    x0, x1 = symbols('x0, x1')
    f = x0**3 - 72*x0 - 220
    f1 = diff(f, x0)
    x1 = x0 - f/f1
    return round(x1.subs(x0, a).evalf(), 4)
while a != abc(a):
    abc(a)
    a = abc(a)
    print(a.round(3))  

10.211
9.756
9.727
9.727


In [59]:
#считаем уравнение
a = 2.2
def abc(a):
    x0, x1 = symbols('x0, x1')
    f = x0**2 + 9*x0 - 5
    f1 = diff(f, x0)
    x1 = x0 - f/f1
    return round(x1.subs(x0, a).evalf(), 4)
while a != abc(a):
    abc(a)
    a = abc(a)
    print(a.round(2))  

0.73
0.53
0.52


In [62]:
#оптимизируем с помощью метода Ньютона
def func1(x):
    return 24*x**2 - 4*x
def func2(x):
    return 48*x - 4
from scipy.optimize import newton
newton(func = func1, fprime = func2, x0=42, tol=0.0001).round(3)

0.167

### квазиньютоновские методы для оптимизации функции

In [63]:
import numpy as np
from scipy.optimize import minimize

In [64]:
#Определим функцию, которую будем оптимизировать
def func(x):
    return x[0]**2.0 + x[1]**2.0

In [65]:
#определим градиент для функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

In [66]:
#Зададим начальную точку
x_0 = [1.0, 1.0]

In [67]:
#определим алгоритм
result = minimize(func, x_0, method='BFGS', jac=grad_func)

In [68]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [69]:
#Можно повторить то же самое с вариацией  L-BFGS-B
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


#### практика

In [77]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20
 
#  определяем градиент функции, пишем две производные
def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])
 
# определяем начальную точку
x_0 = [-400, -400]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution.round(), evaluation.round()))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 9
Решение: f([-4.  1.]) = -1.00000


In [81]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - 3*x[0] + 45
 
#  определяем градиент функции, пишем две производные
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = 10
# реализуем алгоритм BFGS
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 5
Решение: f([1.5]) = 42.75000


In [82]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - 3*x[0] + 45
 
#  определяем градиент функции, пишем две производные
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = 10
# реализуем алгоритм BFGS
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([1.5]) = 42.75000


In [84]:
# определяем нашу функцию
def func(x):
    return x[0]**4 + 6*x[1]**2 + 10
 
#  определяем градиент функции, пишем две производные
def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм BFGS
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Количество оценок: 40
Решение: f([-9.52718297e-03 -2.32170510e-06]) = 10.00000


In [85]:
# определяем нашу функцию
def func(x):
    return x[0]**4 + 6*x[1]**2 + 10
 
#  определяем градиент функции, пишем две производные
def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм BFGS
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000


## Линейное программирование
### SciPy

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [86]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [87]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [88]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

### CVXPY

In [90]:
import cvxpy

In [ ]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [ ]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [91]:
#Укажем его размерность, а также условие,
# что все числа в массиве должны быть целыми
x = cvxpy.Variable(shape=n, integer = True)

In [92]:
#Далее зададим ограничения, используя матричное умножение
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

In [93]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [95]:
#Вызываем получившееся решение:
problem.solve()

SolverError: Solver 'SCIPY' failed. Try another solver, or solve with verbose=True for more information.

In [96]:
#добавляем условие что х положительный
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([-0., -0., -0.,  7., -0.,  0.])

А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём х типа boolean.

In [97]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([1., 0., 0., 1., 0., 1.])

### PuLP

Пример № 3. PuLP

В нашей каршеринговой компании две модели автомобилей: модель  и модель . Автомобиль  даёт прибыль в размере 20 тысяч в месяц, а автомобиль  — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа , и 5 дней — для каждого автомобиля типа .
Заводу требуется 3 дня, чтобы изготовить модель , и 6 дней, чтобы изготовить модель .
Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль , и 7 дней —  автомобиль .
Каждый специалист может работать суммарно 30 дней.

In [99]:
from pulp import *

In [100]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

c:\Python\Python312\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


In [110]:
problem  = LpProblem('Доставка до магазинов', LpMinimize)
A1 = LpVariable('Склад 11', lowBound=0)
A2 = LpVariable('Склад 12', lowBound=0)
A3 = LpVariable('Склад 13', lowBound=0)
M1 = LpVariable('Склад 21', lowBound=0)
M2 = LpVariable('Склад 22', lowBound=0)
M3 = LpVariable('Склад 23', lowBound=0)
problem += A1*2 + A2*5 + A3*3 + M1*7 + M2*7 + M3*6
problem += A1 + A2 + A3 <=180
problem += M1 + M2 + M3 <=220
problem += A1 + M1 >=110
problem += A2 + M2 >=150
problem += A3 + M3 >=140
problem.solve()
print(value(problem.objective))

1900.0


In [116]:
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 100, 4],
              [8, 2, 20, 4, 1000]])
x = cvxpy.Variable(shape=(5,5), boolean=True)
cons = [cvxpy.sum(x, axis=0) == np.ones(5),
        cvxpy.sum(x, axis=1) == np.ones(5)]
func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=cons)
problem.solve()
(x.value*c).sum()

32.0

In [118]:
c = np.array([[0, 12, 10, 19, 8],
             [12, 0, 3, 7, 2],
             [10, 3, 0, 6, 20],
             [19, 7, 6, 0, 4],
             [8, 2, 20, 4, 0]])
x = cvxpy.Variable(shape=(5,5), boolean=True)
cons = [cvxpy.sum(x, axis=0) == np.ones(5),
        cvxpy.sum(x, axis=1) == np.ones(5)]

for i in range(5):
    cons.append(x[i, i] == 0)

for i in range(4):
    for j in range(i + 1, 5):
        cons.append(x[i, j] + x[j, i] <= 1)
        
func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=cons)
problem.solve()
(x.value*c).sum()

32.0